In [1]:
import pandas as pd

In [2]:
filename = 'query_all_org_class_results2.xlsx'
outfile = 'output_class_to_orgs_20190501_v10.xlsx'

filepath = '/Volumes/backup_128G/z_repository/TBIO_data/RequestsFromTana/all_org_classes'
read_file_from = '{0}/{1}'.format(filepath, filename)
write_file_to = '{0}/{1}'.format(filepath, outfile)

In [3]:
df = pd.read_excel(read_file_from)
df = df.fillna(0)
df.head()

,graph,orgName,class
0,yjc,Organization,0
1,yjc,PrivateOrganization,"(公司設在竹東郡蕃地油羅山由台北海山郡及文山部的三井製材所提供原料),製"
2,yjc,Organization,"(公司設在竹東郡蕃地油羅山由台北海山郡及文山部的三井製材所提供原料),製"
3,yjc,Organization,)
4,yjc,Organization,+_後在嘉義設立分店


In [4]:
print(df.count())
count = max(df.count())
print('max count = {0}'.format(count))

graph      35591
orgName    35591
class      35591
dtype: int64
max count = 35591


In [5]:
count = df.groupby(['orgName']).count()['graph']

count = count.sort_values(ascending=False)
print(count)

maxCount = max(count)

orgName
Organization                                 8625
PrivateOrganization                          7834
OrganizationUnit                             4773
PublicOrganization                           3759
StateOrganization                            3753
ElectedOrganization                          1436
FinancialOrganization                        1175
AdministrativeOrganization                    904
HealthOrganization                            683
AgriculturalOrganization                      634
EducationalOrganization                       442
ProvincialOrganization                        286
MonopolyOrganization                          254
EarlyPostWarOrganization                      207
Trade_FinancialOrganization                   168
ProfessionalOrganization                      154
ReligiousOrganization                          87
CommunityOrganization                          68
ForestryOrganization                           65
PartyOrganization                         

In [6]:
results = []

for idx in range(0, len(df)):
    row = df.loc[idx]
#     print(row)
    graph = row['graph']
    orgName = row['orgName']
    className = row['class']
    
    comments = ''
    for cidx in range(3, len(row)):
        if row[cidx] != 0:
            comments += '{0};'.format(row[cidx])
    
    result = {'graph': graph, 'org': orgName, 'class': className, 'comments': comments}
    if result not in results:
        results.append(result)
    
#     print(graph, orgName, className, comments)
print(len(results))
print(results[0:5])

35591
[{'graph': 'yjc', 'org': 'Organization', 'class': 0, 'comments': ''}, {'graph': 'yjc', 'org': 'PrivateOrganization', 'class': '(公司設在竹東郡蕃地油羅山由台北海山郡及文山部的三井製材所提供原料),製', 'comments': ''}, {'graph': 'yjc', 'org': 'Organization', 'class': '(公司設在竹東郡蕃地油羅山由台北海山郡及文山部的三井製材所提供原料),製', 'comments': ''}, {'graph': 'yjc', 'org': 'Organization', 'class': ')', 'comments': ''}, {'graph': 'yjc', 'org': 'Organization', 'class': '+_後在嘉義設立分店', 'comments': ''}]


In [7]:
# classResults = {}

# for res in results:
#     className = res['class']
#     graph = res['graph']
#     orgName = res['org']
#     comments = res['comments']

#     if className not in classResults:
#         classResults[className] = {'orgGra': {}, 'comments': ''}
        
#     orgGraph = classResults[className]['orgGra'].copy()
#     if orgName not in orgGraph:
#         orgGraph[orgName] = []
    
#     if graph not in orgGraph[orgName]:
#         orgGraph[orgName].append(graph)
    
#     classResults[className] = {'orgGra': orgGraph, 'comments': comments}

# print(len(results), len(classResults))

In [8]:
def mapFunc(orgName, defaultVal, mapTable):
    res = defaultVal;
    for tKey in mapTable:
        for tVale in mapTable[tKey]:
            if tVale in orgName:
                return tKey

    return res

## Ownership mapping table

In [9]:
# Ownership mapping table
OwnershipMapping = {
    'PrivateOrganization': ['PrivateOrganization'], 
    'StateOrganization': ['StateOrganization'],
    'PublicOrganization': ['PublicOrganization']
}
UnknownOwnership = '';

## ModeOfEntry mapping table

In [10]:
# ModeOfEntry mapping table
ModeOfEntryMapping = {
    'Appointed': ['總督府','街','庄','州','公學校','利組合','組合'], 
    'Elected': ['協議會']
}
UnknownModeOfEntry = '';

## Period mapping table

In [11]:
## Period mapping table
# 總督府，街，庄，信託會社，州，公學校，株式會，株式会社，利組合，組合，國語講習所
PeriodMapping = {
    'JapaneseColonialOrganization': ['總督府','街','庄','信託會社','州','公學校',
                                     '株式會','株式会社','利組合','組合','國語講習所'],
    'EarlyPostWarOrganization': ['縣議會','台灣省','台灣長官公署'],
    'RepublicanOrganization': [],
}
UnknownPeriod = '';

## yjc 為 JapaneseColonialOrganization


## LevelOfOperation mapping table

In [12]:
## LevelOfOperation mapping table
LevelOfOperationMapping = {
    'CentralOrganization': [],
    'LocalOrganization': ['庄','街','區','州','市','縣'],
    'ProvincialOrganization': ['省','廳']
}
UnknownLevelOfOperation = '';

## Function mapping table

In [13]:
## Function mapping table
func_filename = 'query_org_divisionContent_results.csv'
func_file_from = '{0}/{1}'.format(filepath, func_filename)

funcDf = pd.read_csv(func_file_from)
funcDf.head()

def mapOrgToFunction(orgName, defaultVal):
#     print(orgName)
    row = funcDf.loc[funcDf['sName'] == orgName]
    if row.empty:
        division = [defaultVal]
    else:
        division = row['orgName'].tolist()

    return division

# UnknownFunction = 'UnknwownFunction';
# print(mapOrgToFunction('壯圍庄古亭笨第十五保第七保保甲', UnknownFunction))
# print(mapOrgToFunction('高雄州農會', UnknownFunction))
# print(mapOrgToFunction('在魚池庄經營製茶業', UnknownFunction))
# print(mapOrgToFunction('xxxxx', UnknownFunction))

# for idx in range(0, len(funcDf.index)):
# # for idx in range(16,23):
# #     print(funcDf.loc[idx, 'sName'])
#     print(mapOrgToFunction(funcDf.loc[idx, 'sName'], UnknownFunction))


In [14]:
# Function mapping table
FunctionMapping = {    
    'ArmedForces': ['ArmedForces'],
    'PrivateBusiness': ['AgriculturalOrganization', 'Industry', 'Business', 'Trade_FinancialOrganization'],
    'PublicAdministration': ['SocialAffairs', 'AdministrativeOrganization', 'CommunicationPost', 
                             'Hygiene_Sanitation', 'Canals_Irrigation', 'Transportation',
                             'JudicialOrganization'],
    'Academia_Education': ['EducationalOrganization', 'Research_Investigation'],
    'Hospital': ['HealthOrganization'],
    'VoluntaryAssociation': ['ReligiousOrganization'],
    'StateBusiness': ['MonopolyOrganization'],
    'MassMedia': ['Media'],
    'NeedToBeSolved': ['EntertainmentOrganization']
}

UnknownFunction = '';

## Main generation

In [32]:
print(len(results))

35591


In [15]:
# orgList = []
# # [{'graph': 'chrj_pr', 'org': 'Organization', 'class': '大興紡織', 'comments': ''},
# for res in results:
#     className = str(res['class'])
#     graph = res['graph']
#     orgName = res['org']

#     orgArr = []

#     # graph
#     orgArr.append(graph)
    
#     # Organization
#     orgArr.append(className)

#     # Ownership
# #     orgArr.append(mapFunc(orgName, UnknownOwnership, OwnershipMapping))

#     # ModeOfEntry
# #     orgArr.append(mapFunc(className, UnknownModeOfEntry, ModeOfEntryMapping))

#     # Period
#     periodTitle = mapFunc(className, UnknownPeriod, PeriodMapping)
#     if periodTitle == UnknownPeriod and graph == 'yjc':
#         periodTitle = 'JapaneseColonialOrganization'
    
#     orgArr.append(periodTitle)

#     # LevelOfOperation
# #     orgArr.append(mapFunc(className, UnknownLevelOfOperation, LevelOfOperationMapping))

#     # Function
#     funcArray = mapOrgToFunction(className, UnknownFunction)
# #         orgArr.append(mapOrgToFunction(strClassName, UnknownFunction))

#     # comments
# #     comments = res['comments']
        
#     for funcName in funcArray:
#         finalOrg = []
#         finalOrg = orgArr.copy()
#         finalOrg.append(mapFunc(funcName, UnknownFunction, FunctionMapping))
# #         finalOrg.append(comments)

#         if finalOrg not in orgList:
#             orgList.append(finalOrg)
# #         else:
# #             print("Duplicated", finalOrg)


In [16]:
# print("results", len(results))
# print("orgList", len(orgList))
# print(orgList[0:5])

results 35591
orgList 22348
[['yjc', '0', 'JapaneseColonialOrganization', ''], ['yjc', '(公司設在竹東郡蕃地油羅山由台北海山郡及文山部的三井製材所提供原料),製', 'JapaneseColonialOrganization', 'PrivateBusiness'], ['yjc', ')', 'JapaneseColonialOrganization', ''], ['yjc', '+_後在嘉義設立分店', 'JapaneseColonialOrganization', ''], ['yjc', '+_獲特選第一名及台展賞', 'JapaneseColonialOrganization', '']]


## Merge what Meng-lun has done.

In [24]:
merge_file_name = 'output_class_to_orgs_20190501_v10_tana.xlsx'
merge_file = '{0}/{1}'.format(filepath, merge_file_name)

In [25]:
mergeDf = pd.read_excel(merge_file)
mergeDf = mergeDf.fillna(0)
mergeDf.head()

,Organization,SocietalSector,Period,Correct name
0,上海真如學院,Academia_Education,JapaneseColonialOrganization,0
1,下關商業學校,Academia_Education,JapaneseColonialOrganization,0
2,中埔公學校白芒埔分教場,Academia_Education,JapaneseColonialOrganization,0
3,中埔公學校頂六分教場,Academia_Education,JapaneseColonialOrganization,0
4,中壢皇民奉公會,PublicAdministration,JapaneseColonialOrganization,0


In [46]:
mergeResults = []
for idx in range(0, len(mergeDf)):
    row = mergeDf.loc[idx]
#     print(row)

    organization = str(row['Organization'])
    
    # find the graph of the organization
    graph = ''
    for res in results:
        resGraph = res['graph']
        resOrg = res['class']
    
        if resOrg == organization and resGraph == 'chrj_pr':
            break
            
    mergeResults.append([organization, graph])
#     societalSector = str(row['SocietalSector'])
#     period = str(row['Period'])
#     corrName = str(row['Correct name'])
    
#     if organization not in mergeResults:
#         mergeResults[organization] = {'societalSector': societalSector, 'period': period, 'Correct': corrName}
    
#     if societalSector != '0':
#         mergeResults[organization]['societalSector'] = societalSector
#     if period != '0':
#         mergeResults[organization]['period'] = period
#     if comments != '0':
#         mergeResults[organization]['comments'] = comments
        
# mergeResults

In [47]:
mergeResults[0:5]

[['上海真如學院', ''],
 ['下關商業學校', ''],
 ['中埔公學校白芒埔分教場', ''],
 ['中埔公學校頂六分教場', ''],
 ['中壢皇民奉公會', '']]

In [48]:
print(len(mergeResults))

19537


In [49]:
orgDf = pd.DataFrame(mergeResults, columns=['org', 'graph'])
orgDf.head()

,org,graph
0,上海真如學院,
1,下關商業學校,
2,中埔公學校白芒埔分教場,
3,中埔公學校頂六分教場,
4,中壢皇民奉公會,


In [50]:
write_orgGraph_file = '{0}/{1}'.format(filepath, 'orgGraphFile.xlsx')
orgDf.to_excel(write_orgGraph_file)

In [21]:
# mergeList = []
# for org in orgList:
#     orgName = org[0]
#     periodSuggest = org[1]
#     societalSuggest = org[2]
    
#     merge = []
#     merge.append(orgName)
    
#     comments = ''
#     if orgName in mergeResults:
#         societal = mergeResults[orgName]['societalSector']
#         if societal == '0':
#             societal = ''
#         merge.append(societal)
        
#         period = mergeResults[orgName]['period']
#         if period == '0':
#             period = ''
#         merge.append(period)
        
#         comments = mergeResults[orgName]['comments']
#         if comments == '0':
#             comments = ''

#     else:
#         merge.append('')
#         merge.append('')
        
#     merge.append(societalSuggest)
#     merge.append(periodSuggest)
#     merge.append(comments)
    
#     mergeList.append(merge)
        
# print(len(mergeList), mergeList[0:5])

22348 [['yjc', '', '', 'JapaneseColonialOrganization', '0', ''], ['yjc', '', '', 'JapaneseColonialOrganization', '(公司設在竹東郡蕃地油羅山由台北海山郡及文山部的三井製材所提供原料),製', ''], ['yjc', '', '', 'JapaneseColonialOrganization', ')', ''], ['yjc', '', '', 'JapaneseColonialOrganization', '+_後在嘉義設立分店', ''], ['yjc', '', '', 'JapaneseColonialOrganization', '+_獲特選第一名及台展賞', '']]


## Output file

In [22]:
# def getMaxLen(orgList):
#     maxEleLen = 0
#     for org in orgList:
#         orgLen = len(org)
#         if maxEleLen < orgLen:
#             maxEleLen = orgLen
#     return maxEleLen

# print(getMaxLen(mergeList))
# # columns = ['Organization','Ownership','ModeOfEntry','Period','LevelOfOperation','SocietalSector','comments']  
# columns = ['Organization','SocietalSector','Period','SocietalSectorSuggest','PeriodSuggest','comments']  
# columns

6


['Graph',
 'Organization',
 'SocietalSector',
 'Period',
 'SocietalSectorSuggest',
 'PeriodSuggest',
 'comments']

In [23]:
# orgDf = pd.DataFrame(mergeList, columns=columns)
# orgDf.drop_duplicates(subset='Organization', keep='first', inplace=True)
# orgDf.sort_values(by=['Organization', 'PeriodSuggest'], inplace=True)
# # orgDf = orgDf.fillna('')
# orgDf.head()

AssertionError: 7 columns passed, passed data had 6 columns

In [ ]:
# orgDf.to_excel(write_file_to, index=False)

In [ ]:
# # double check the duplicated
# duplicateRowsDF = orgDf[orgDf.duplicated(['Organization'], keep='last')]
# duplicateRowsDF